In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine

In [3]:
# Note:: The make sure you use the information from your specific PostgreSQL installation
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'Staver71300!' # this is the password established during installation
port = r'5432' # default port estabalished during install

In [4]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [5]:
table_name = r'planets_data'
schema = r'raw' # schema were the data was loaded last week.

df = pd.read_sql_table(table_name, db_conn, schema)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


In [7]:
df.head(10)

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009
5,Radial Velocity,1,185.840,4.80,76.39,2008
6,Radial Velocity,1,1773.400,4.64,18.15,2002
7,Radial Velocity,1,798.500,NaN,21.41,1996
8,Radial Velocity,1,993.300,10.30,73.10,2008
9,Radial Velocity,2,452.800,1.99,74.79,2010


In [8]:
clean_planets = df.copy()

In [23]:
clean_planets.isna().sum()

method              0
number              0
orbital_period     43
mass              522
distance          227
year                0
dtype: int64

In [26]:
clean_planets = clean_planets.dropna(subset=['orbital_period'])

In [27]:
clean_planets.isna().sum()

method              0
number              0
orbital_period      0
mass              479
distance          216
year                0
dtype: int64

Entries with an empty orbital_period value are the only entried we can drop without erasing over 20% of the data.

In [28]:
clean_planets['gravitation'] = clean_planets['mass'] / clean_planets['distance']

In [29]:
clean_planets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 992 entries, 0 to 1034
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          992 non-null    object 
 1   number          992 non-null    int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        776 non-null    float64
 5   year            992 non-null    int64  
 6   gravitation     498 non-null    float64
dtypes: float64(4), int64(2), object(1)
memory usage: 62.0+ KB


In [31]:
clean_planets.head(15)

,method,number,orbital_period,mass,distance,year,gravitation
0,Radial Velocity,1,269.300,7.10,77.40,2006,0.091731
1,Radial Velocity,1,874.774,2.21,56.95,2008,0.038806
2,Radial Velocity,1,763.000,2.60,19.84,2011,0.131048
3,Radial Velocity,1,326.030,19.40,110.62,2007,0.175375
4,Radial Velocity,1,516.220,10.50,119.47,2009,0.087888
5,Radial Velocity,1,185.840,4.80,76.39,2008,0.062835
6,Radial Velocity,1,1773.400,4.64,18.15,2002,0.255647
7,Radial Velocity,1,798.500,NaN,21.41,1996,NaN
8,Radial Velocity,1,993.300,10.30,73.10,2008,0.140903
9,Radial Velocity,2,452.800,1.99,74.79,2010,0.026608


Gravitation is a made up variable that gives an arbitrary number representing the relationship between distance from the earth and mass, two variables that are used in determinig the gravitational pull that the distant planets would have on the earth. 

### Time to load some data
We need to define a name for the table that we are above to create in our database.

In [32]:
table_name = r'planets_cleaned'
schema = r'cleaned' # schema we created this week.

clean_planets.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

992